# Backprop seminar

### by Talgat

## The Importance of Understanding Backpropagation

Understanding backpropagation is crucial for both researchers and practitioners in the field of deep learning for different reasons:

- **For Researchers**: The ability to write custom layers and define their differentiation process is essential.

- **For Practitioners**: Questions on backpropagation are very popular on deep learning interviews.

PyTorch is very similar to NumPy, but these are key differences:
- **GPU Support** PyTorch tensors can be easily moved to a GPU for accelerated computing. See `device` argument and method `.to(device)`. 

- **Automatic Differentiation (and Dynamic Computational Graphs)** PyTorch provides a powerful autograd system that automatically computes gradients for tensor operations, making it easy to train neural networks.



This guide assumes you have a basic understanding of PyTorch. **If you're not familiar**, consider reading the [Deep Learning with PyTorch: A 60 Minute Blitz](https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html) tutorial on the official PyTorch website.

I will cover the most important functions for a custom backpropagation. 

# Gradients in torch

In [42]:
import torch

In [48]:
A = torch.randn(5, 7, dtype=torch.double, requires_grad=True)
norm_value = torch.linalg.norm(A) ** 2
norm_value.backward()
A.grad

tensor([[ 1.9793, -1.0864,  3.1290, -0.3812,  0.8100, -0.7537, -0.6821],
        [-0.4295, -0.2838,  0.8261,  0.3421, -1.6663, -4.6581,  0.3815],
        [-0.3176, -2.2620,  2.0672, -1.0792, -2.6879, -2.4325, -2.1618],
        [-0.4634, -1.0539, -0.7558,  4.3560, -2.1569,  0.8904,  0.9192],
        [ 1.2624,  0.2435,  2.3492, -1.5918, -1.8435,  1.1442, -0.4143]],
       dtype=torch.float64)

In [49]:
torch.allclose(A.grad, 2 * A)

True

In [10]:
# # Error:
# A = torch.randn(5, 7, dtype=torch.double)
# norm_value = torch.linalg.norm(A) ** 2
# norm_value.backward()
# A.grad

In [1]:
# # Error:
# with torch.no_grad():
#     A = torch.randn(5, 7, dtype=torch.double, requires_grad=False)
#     norm_value = torch.linalg.norm(A) ** 2
#     norm_value.backward()
#     A.grad

In [2]:
# # Error:
# A = torch.randn(5, 7, dtype=torch.double, requires_grad=False)
# norm_value = A ** 2
# norm_value.backward()

# Reminder on Backpropagation

# Einstein summation

`torch.einsum(equation_string, *tensors)`:

- `equation_string`: A string that specifies the operation in the form `indices1,...,indices_n->indices_res`

- `tensors`: The tensors to apply the operation to.

Matrix product:
$$
(A B)_{ij} = \sum_{k} a_{ik} b_{kj}
$$

In [12]:
A = torch.rand(3, 5)
B = torch.rand(5, 4)
matmul = torch.einsum('ik,kj->ij', A, B)

In NumPy and PyTorch, only a single symbol can be an index. For more complex cases, use [einops library](https://einops.rocks/).

# TL;DR Custom Gradients with Backpropagation

Compute custom gradients for a function $f: \mathbb{R}^n \to \mathbb{R}^m$ as follows:

1. **Think of a Scalar Function** $g: \mathbb{R}^m \to \mathbb{R}$, that maps $f$'s output to a scalar.

2. **Write Chain Rule**: The derivative of $g$ with respect to $x_i$ is:

   $$
   \frac{\partial}{\partial x_i} g(f(x)) = \sum_{k} \frac{\partial g(f(x))}{\partial (f(x)_k)} \frac{\partial (f(x)_k)}{\partial x_i}
   $$

   - The term $\frac{\partial g(f(x))}{\partial (f(x)_k)}$ is provided (gradient from a subsequent layer). We will call it `grad_output`.
   - The second term is the Jacobian of your function. 

3. **Perform Vector-Jacobian Product** (vJp): Multiply the given gradient vector by $f$'s Jacobian. 


The result is equal to `einsum('k,ki->i', vector, Jacobian)`, though direct computation of vJp is usually not efficient.

# Matvec by vector

Suppose $A \in \mathbb{R}^{m \times n}, x \in \mathbb{R}^{n}$, $f(A, x) = Ax$. 

And suppose $g$ is some function of $f(A, x)$. It can be multidimensional, but without loss of generality we consider one-dimensional case, e.g. $g(\cdot) = \| \cdot \|^2$. (Vector and matrix functions can be differenciated in the same way.)

Chain-rule for $x$:

$$
\dfrac{\partial}{\partial x_{i}} g(f(A, x))
    =
\sum\limits_{k}
    \underbrace{\dfrac{\partial g(Ax)}{\partial (Ax)_k}}_\text{grad_output[k]}
\cdot
    \underbrace{\dfrac{\partial (Ax)_k}{\partial x_i}}_{\nabla_x (Ax)[k][i]}
    =
\sum\limits_{k}
\underbrace{\dfrac{\partial g(Ax)}{\partial (Ax)_k}}_\text{grad_output[k]}
    \cdot
a_{ki}
    =
A^\top \text{grad_output}
$$ 

In einsum notation, `einsum('k,ki->i', grad_output, dAx_dx)`

# Matvec by matrix

Gradients w.r.t. A:
$$
\dfrac{\partial}{\partial a_{ij}} g(f(A, x))
    =
\sum\limits_{k}
    \underbrace{\dfrac{\partial g(Ax)}{\partial (Ax)_k}}_\text{$g_k :=$ grad_output[k]}
\cdot
    \underbrace{\dfrac{\partial (Ax)_k}{\partial a_{i, j}}}_{J_{ijk}}
$$

It is easy to understand that $J_{ijk} = x_j \cdot [k = i]$, where $[\cdot]$ is an indicator function. That's why
$$
\sum\limits_{k} g_k f_{i, j, k} = g_i^\top x_j.
$$

In einsum notation, `einsum('k,ijk->ij', grad_output, dAx_dA)`

## Custom Gradients in PyTorch

### Autograd Functions

- **Purpose**: Enable fine-grained control over both the forward and backward computations.
- **Usage**: Define a class inheriting from `torch.autograd.Function` and implement the `@staticmethod`s `forward` and `backward`.

```python
from torch.autograd import Function

class CustomFunction(Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return output

    @staticmethod
    def backward(ctx, grad_output):
        input, = ctx.saved_tensors
        grad_input = ...
        return grad_input


# Custom gradients in PyTorch

In [67]:
import torch
from torch.autograd import gradcheck

class MyMatVec(torch.autograd.Function):
    @staticmethod
    def forward(ctx, A, x):
        ctx.save_for_backward(A, x)
        return torch.mv(A, x)

    @staticmethod
    def backward(ctx, grad_output):
        A, x = ctx.saved_tensors  # Retrieve saved tensors
        grad_A = grad_output[:, None] @ x[None, :]
        grad_x = A.T @ grad_output
        return grad_A, grad_x

# Alias for applying the operation
matvec = MyMatVec.apply

# Example usage
A = torch.randn(5, 7, dtype=torch.double, requires_grad=True)
x = torch.randn(7, dtype=torch.double, requires_grad=True)
inputs = (A, x)
if gradcheck(matvec, inputs, eps=1e-6, atol=1e-4):
    print('OK!')


OK!


# Backward in pytorch

In [14]:
A = torch.randn(5, 7, dtype=torch.double, requires_grad=True)
norm_value = torch.linalg.norm(A) ** 2
norm_value.backward()
A.grad

tensor([[ 0.0142, -3.2544, -3.9560,  5.5219,  1.1941, -0.7747, -0.4376],
        [ 0.8601, -0.0065, -0.4173,  2.2674,  0.7285, -0.0284, -2.1073],
        [-1.3998, -0.6446, -1.2715,  0.3235,  1.3581, -3.5613, -1.6263],
        [-1.6602, -1.1135,  0.6004,  1.4760,  2.3580, -3.8255, -2.0487],
        [-3.1986, -1.6538, -0.4544, -0.5043,  2.9867, -0.9459,  3.6030]],
       dtype=torch.float64)

In [20]:
# # Error:
# A = torch.randn(5, 7, dtype=torch.double, requires_grad=False)
# norm_value = torch.linalg.norm(A) ** 2
# norm_value.backward()
# A.grad

In [22]:
# # Error:
# with torch.no_grad():
#     A = torch.randn(5, 7, dtype=torch.double, requires_grad=False)
#     norm_value = torch.linalg.norm(A) ** 2
#     norm_value.backward()
#     A.grad

## Dropout

During training:

$\text{dropout}(x) = \dfrac{1}{1 - p} \text{Bernoulli}(1 - p) \odot x$,

where $\odot$ is an element-wise product, $\text{Bernoulli}(1 - p)$ is a random binary mask from Bernoulli distribution.

During inference do nothing. 

In [16]:
import torch
from torch.autograd import gradcheck

class MyDropOutTraining(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, mask, p=0.1):
        res = x * mask / (1 - p)
        ctx.save_for_backward(mask / (1 - p))
        return res

    @staticmethod
    def backward(ctx, grad_output):
        scaled_mask, = ctx.saved_tensors
        return scaled_mask * grad_output, None, None

# Alias for applying the operation
dropout = MyDropOutTraining.apply


# Example usage
x = torch.randn(7, dtype=torch.double, requires_grad=True)
p = 0.1
mask = torch.empty_like(x)
mask.bernoulli_(1 - p) 
inputs = (x, mask, p)
# Gradcheck will not work with a mask that is not an input
if gradcheck(dropout, inputs, eps=1e-6, atol=1e-4):
    print('OK!')

OK!


## SoftMax

Softmax converts an arbitrary vector to a vector that sums up to one. It is a typical last layer for multiclass classification problems, where we have to output the probability. 

$$
\text{softmax}(x)_i = \frac{\exp{x_i}}{\sum_{k} \exp{x_k}}, \quad x \in \mathbb{R}^n
$$

**Exercise** Implement custom gradients for this function.



In [100]:
import torch
from torch.autograd import gradcheck

class MySoftMax(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x):
        x = x - x.max()  # for stability
        exp_vals = torch.exp(x)
        res = exp_vals / torch.sum(exp_vals)
        ctx.save_for_backward(x, res)
        return res

    @staticmethod
    def backward(ctx, grad_output):
        x, res = ctx.saved_tensors  
        # Your code
        return torch.ones_like(x)

# Alias for applying the operation
softmax = MySoftMax.apply

# Example usage
x = torch.randn(7, dtype=torch.double, requires_grad=True)
inputs = x
if gradcheck(softmax, inputs, eps=1e-6, atol=1e-4):
    print('OK!')

In [17]:
f = torch.randn(5)

torch.einsum('i,j->ij', f, f)

tensor([[ 0.3448,  0.0757, -0.8317,  0.9454, -0.2449],
        [ 0.0757,  0.0166, -0.1825,  0.2074, -0.0537],
        [-0.8317, -0.1825,  2.0063, -2.2806,  0.5908],
        [ 0.9454,  0.2074, -2.2806,  2.5924, -0.6716],
        [-0.2449, -0.0537,  0.5908, -0.6716,  0.1740]])

In [18]:
f[:, None] @ f[None, :]

tensor([[ 0.3448,  0.0757, -0.8317,  0.9454, -0.2449],
        [ 0.0757,  0.0166, -0.1825,  0.2074, -0.0537],
        [-0.8317, -0.1825,  2.0063, -2.2806,  0.5908],
        [ 0.9454,  0.2074, -2.2806,  2.5924, -0.6716],
        [-0.2449, -0.0537,  0.5908, -0.6716,  0.1740]])

In [19]:
f.reshape(-1, 1) @ f.reshape(1, -1)

tensor([[ 0.3448,  0.0757, -0.8317,  0.9454, -0.2449],
        [ 0.0757,  0.0166, -0.1825,  0.2074, -0.0537],
        [-0.8317, -0.1825,  2.0063, -2.2806,  0.5908],
        [ 0.9454,  0.2074, -2.2806,  2.5924, -0.6716],
        [-0.2449, -0.0537,  0.5908, -0.6716,  0.1740]])

# Conv1D

$g(x, w)[i] = \sum_{j=0}^{k-1} w[j] \cdot x[i+j], \quad x \in \mathbb{R}^n, \ w \in \mathbb{R}^k, \ g(x, w) \in \mathbb{R}^{n - k + 1}$

Write the backward formula...

# Example: torchdiffeq

In [20]:
! pip install torchdiffeq -q

In [22]:
import torch
import torch.nn as nn
from torchdiffeq import odeint_adjoint as odeint

# Define the ODE system (function) to be solved
class SimpleODEFunc(nn.Module):
    def __init__(self):
        super().__init__()
        # Parameters or any other components can be defined here if needed

    def forward(self, t, y):
        # Example ODE: dy/dt = -y
        dydt = -y
        return dydt

# Initial condition
y0 = torch.tensor([[1.0]], requires_grad=True)

# Time points to solve the ODE at (from t=0 to t=1)
t = torch.linspace(0, 1, steps=100)

# Instantiate the ODE function
func = SimpleODEFunc()

# Solve the ODE
y = odeint(func, y0, t)

y = torch.linalg.norm(y)
y.backward()


Deep Equilibrium Models: https://implicit-layers-tutorial.org/deep_equilibrium_models/